In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import scipy.stats as scs
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_val_score

In [2]:
# Data from: https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe

In [3]:
data = pd.read_csv('Hotel_Reviews.csv')

In [4]:
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [6]:
for i in data.columns:
    print(i, len(data[i].unique()))

Hotel_Address 1493
Additional_Number_of_Scoring 480
Review_Date 731
Average_Score 34
Hotel_Name 1492
Reviewer_Nationality 227
Negative_Review 330011
Review_Total_Negative_Word_Counts 402
Total_Number_of_Reviews 1142
Positive_Review 412601
Review_Total_Positive_Word_Counts 365
Total_Number_of_Reviews_Reviewer_Has_Given 198
Reviewer_Score 37
Tags 55242
days_since_review 731
lat 1473
lng 1473


In [7]:
# remove rows with no review
data = data[(data['Negative_Review'] != 'No Negative') | (data['Positive_Review'] != 'No Positive')]

In [ ]:
data.drop('Hotel_Name', axis=1, inplace=True)
data['Average_Score'].astype(float)

In [ ]:
# Benchmark - error if you always guessed the average
mean_absolute_error(data['Average_Score'], data['Reviewer_Score'])

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
tags = data.Tags

In [ ]:
import ast
new_tags = []
for i in tags:
    new_tags.append(ast.literal_eval(i))

In [ ]:
# check most common tags
from collections import Counter
list_for_counting = []
for i in new_tags:
    for t in i:
        list_for_counting.append(t)
    
c = Counter(list_for_counting)
c.most_common(20)      

In [ ]:
# feature engineering tags to reduce columns
for index, i in enumerate(new_tags):
    for idx, t in enumerate(i):
        if t in [' Stayed 1 night ',' Stayed 2 nights ']:
            new_tags[index][idx] = 'Stayed 1-2 nights'
        if t in [' Stayed 3 nights ',' Stayed 4 nights ']:
            new_tags[index][idx] = 'Stayed 3-4 nights'
        if t in [' Stayed 5 nights ',' Stayed 6 nights ', ' Stayed 7 nights ', ' Stayed 8 nights ', ' Stayed 9 nights ', ' Stayed 10 nights ',  ' Stayed 11 nights ',
             ' Stayed 12 nights ', ' Stayed 13 nights ', ' Stayed 14 nights ', ' Stayed 15 nights ', ' Stayed 16 nights ', ' Stayed 17 nights ',' Stayed 18 nights ', ' Stayed 19 nights ', ' Stayed 20 nights ',
             ' Stayed 21 nights ', ' Stayed 22 nights ', ' Stayed 23 nights ', ' Stayed 24 nights ', ' Stayed 25 nights ', ' Stayed 26 nights ',
             ' Stayed 27 nights ', ' Stayed 28 nights ', ' Stayed 29 nights ', ' Stayed 30 nights ', ' Stayed 31 nights ',]:
            new_tags[index][idx] = 'Stayed 5+ nights'
        if 'Apartment' in t or 'Apartement' in t:
            new_tags[index][idx] = 'Apartment'
        if 'Luxury' in t or 'VIP' in t or 'Executive' in t or 'Ambassador' in t or 'Royal' in t or 'Penthouse' in t or 'Suite' in t:
            new_tags[index][idx] = 'Fancy'
        if 'Standard' in t or 'Budget' in t or 'Small' in t or 'Economy' in t or 'Basic' in t or 'Bunk Bed' in t or 'Interior' in t or 'Special Offer' in t:
            new_tags[index][idx] = 'Budget'
        if 'Comfort' in t or 'Family' in t or 'Classic' in t or 'Large' in t:
            new_tags[index][idx] = 'Medium'
        if 'Superior' in t or 'Premium' in t or 'Prestige' in t or 'Premiere' in t or 'Privilege' in t or 'Deluxe' in t or 'Premier' in t or 'Club' in t:
            new_tags[index][idx] = 'High'
tags_final = []
for index, i in enumerate(new_tags):
    new = []
    for idx, t in enumerate(i):
        if t in ['Stayed 1-2 nights','Stayed 3-4 nights', 'Suite', 'Apartment', 'Fancy', 'Budget', 'Medium', 'High', ' Business Trip ', ' Solo Traveler ', ' Leisure trip ', ' Couple ', ' Group ', ' Family with young children ', ' Family with older children ']:
            new.append(i[idx])
    tags_final.append(new)

In [ ]:
len(tags_final)

In [ ]:
unique_tags = []
for i in tags_final:
    for t in i:
        if t not in unique_tags:
            unique_tags.append(t)

In [ ]:
len(unique_tags)

In [ ]:
sorted(unique_tags)

In [ ]:
data['New_Tags'] = tags_final

In [ ]:
len(tags_final)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

tagdf = pd.DataFrame(mlb.fit_transform(data.New_Tags),columns=mlb.classes_, index=data.index)


data = data.join(tagdf)
data = data.drop(['New_Tags'], axis=1)

In [ ]:
data

In [ ]:
#turn object into in
data['days_since_review'] = data['days_since_review'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
data.drop('Hotel_Address', axis=1, inplace=True)

In [ ]:
data.drop('Tags', axis=1, inplace=True)

In [ ]:
neg = data.pop('Negative_Review')
pos = data.pop('Positive_Review')

In [ ]:
data.info()

In [ ]:
nat = pd.get_dummies(data['Reviewer_Nationality'], prefix='Nationality: ')
data = data.join(nat)
data = data.drop(['Reviewer_Nationality'], axis=1)


In [ ]:
data.drop('Review_Date', axis=1, inplace=True)

In [ ]:
data.dtypes[:20]

In [ ]:
lat = data.pop('lat')

In [ ]:
lng = data.pop('lng')

In [ ]:
from textblob import TextBlob

In [ ]:
neg_review_sentiment = []
for i in neg:
    review = TextBlob(i)
    neg_review_sentiment.append(review.sentiment)


In [ ]:
neg_review_sentiments = [i[0] for i in neg_review_sentiment]
data['neg_review_sentiment'] = neg_review_sentiments

In [ ]:
clean = pd.read_csv('cleaned.csv')

In [ ]:
pos_review_sentiment = []
for i in pos:
    review = TextBlob(i)
    pos_review_sentiment.append(review.sentiment)

In [ ]:
pos_review_sentiments = [i[0] for i in pos_review_sentiment]

In [ ]:
clean['pos_review_sentiment'] = pos_review_sentiments

In [ ]:
clean = clean.to_csv('clean.csv')

In [8]:
clean = pd.read_csv('clean.csv')

In [9]:
clean

,Unnamed: 0,Unnamed: 0.1,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,days_since_review,...,Nationality: _ Vanuatu,Nationality: _ Vatican City,Nationality: _ Venezuela,Nationality: _ Vietnam,Nationality: _ Wallis and Futuna,Nationality: _ Yemen,Nationality: _ Zambia,Nationality: _ Zimbabwe,neg_review_sentiment,pos_review_sentiment
0,0,0,194,7.7,397,1403,11,7,2.9,0,...,0,0,0,0,0,0,0,0,0.028671,0.283333
1,1,1,194,7.7,0,1403,105,7,7.5,0,...,0,0,0,0,0,0,0,0,0.150000,0.241960
2,2,2,194,7.7,42,1403,21,9,7.1,3,...,0,0,0,0,0,0,0,0,0.032653,0.460000
3,3,3,194,7.7,210,1403,26,1,3.8,3,...,0,0,0,0,0,0,0,0,-0.070370,0.625000
4,4,4,194,7.7,140,1403,8,3,6.7,10,...,0,0,0,0,0,0,0,0,-0.009091,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515606,515606,515733,168,8.1,14,2823,2,8,7.0,704,...,0,0,0,0,0,0,0,0,0.000000,0.000000
515607,515607,515734,168,8.1,11,2823,11,12,5.8,712,...,0,0,0,0,0,0,0,0,-0.250000,0.250000
515608,515608,515735,168,8.1,19,2823,0,3,2.5,715,...,0,0,0,0,0,0,0,0,0.100000,-0.113636
515609,515609,515736,168,8.1,0,2823,25,3,8.8,717,...,0,0,0,0,0,0,0,0,0.150000,0.300000


In [10]:
y = data.pop('Reviewer_Score')


In [11]:
X = clean
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)

In [ ]:
rf = RandomForestRegressor(min_samples_leaf = 30,
                          max_depth=25,
                          max_features=10,
                          n_estimators=500)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
gdbr = GradientBoostingRegressor(learning_rate=0.01,
                                  loss='ls',
                                 max_depth=15,
                                  n_estimators=100,
                                 min_samples_leaf=120,
                                 max_features=10,
                                  random_state=1)

In [ ]:
gdbr.fit(X_train, y_train)

In [ ]:
y_hat = gdbr.predict(X_test)
mean_absolute_error(y_test, y_hat)

In [ ]:
rf = RandomForestRegressor()
param_dist = dict(min_samples_leaf = [5,10,15,20,30,50],
                          max_depth = [5,10,15,20,25,30],
                          max_features=[10, 20, 30, 40, 50],
                          n_estimators=[50,75,100,300,500,1000])
rand2 = RandomizedSearchCV(rf, param_dist, cv=10, scoring='neg_mean_absolute_error', n_iter=10)

In [ ]:
rand2.fit(X_train, y_train)

from pprint import pprint
pprint(rand2.best_estimator_.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 30,
 'max_features': 40,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

#Best random forest results
RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=25,
                                                   max_features=10,
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=30,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=500,
                                                   n_jobs=None, oob_score=False,
                                                   ra...rbose=0,
                                                   warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': [10, 20, 30, 40, 50],
                                        'min_samples_leaf': [5, 10, 15, 20, 30,
                                                             50],
                                        'n_estimators': [50, 75, 100, 300, 500,
                                                         1000]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='neg_mean_absolute_error',
                   verbose=0)

In [ ]:
yrand2 = rand2.predict(X_test)
mean_absolute_error(y_test, yrand2)

In [12]:
gdbr = GradientBoostingRegressor()
param_dist = dict(min_samples_leaf = [10,15,20,30,50],
                          max_depth = [5,10,20,30],
                          max_features=[10, 20, 30, 40],
                          n_estimators=[50,100,300,500],
                         learning_rate = [0.1, .05, 0.07, 0.01])
booster2 = RandomizedSearchCV(gdbr, param_dist, cv=10, scoring='neg_mean_absolute_error', n_iter=10)

In [13]:
booster2.fit(X_train, y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                            

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.01,
                                                       loss='ls', max_depth=15,
                                                       max_features=10,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=120,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators=100...
                                                       warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': [0.1, 0.05, 0.07,
                                                          0.01],
                                        'max_depth': [5, 10, 20, 30],
                                        'max_features': [10, 20, 30, 40],
                                        'min_samples_leaf': [10, 15, 20, 30,
                                                             50],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='neg_mean_absolute_error',
                   verbose=0)

In [14]:
yboost2 = booster2.predict(X_test)
mean_absolute_error(y_test, yboost2)

0.07593062281089308

#Best gradient boost score
from pprint import pprint
pprint(booster2.best_estimator_.get_params())
{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 30,
 'max_features': 40,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 50,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
pos_X_train, pos_X_test, pos_y_train, pos_y_test = train_test_split(pos,y, test_size=.2)

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(pos_X_train)

In [ ]:
len(vectorizer.get_feature_names())

In [17]:
y.unique()

array([ 2.9,  7.5,  7.1,  3.8,  6.7,  4.6, 10. ,  6.5,  7.9,  5.8,  9.2,
        8.8,  6.3,  5.4,  9.6,  8.3,  4.2,  3.3,  5. ,  2.5,  3.1,  6. ,
        5.5,  9.5,  8. ,  8.5,  9. ,  4.5,  7. ,  5.6,  8.1,  6.9,  9.4,
        3. ,  4. ,  3.5,  4.4])

In [19]:
y.describe()

count    515611.000000
mean          8.395103
std           1.637852
min           2.500000
25%           7.500000
50%           8.800000
75%           9.600000
max          10.000000
Name: Reviewer_Score, dtype: float64

In [ ]:
data.lat

In [ ]:
coordinates = []
for i, j in zip(data.lat, data.lng):
    coordinates.append((i,j))

In [ ]:
type(coordinates[0])

In [ ]:
import folium


m = folium.Map(location=[52.360576, 4.915968])

In [ ]:
for i in range(0, len(coordinates[0])):
    folium.Marker(coordinates[0], popup=df_counters['Name'][point]).add_to(m)
m